In [1]:
from pathlib import Path
from pyprojroot import here as get_project_root
import pandas as pd
import user_config
import openrouteservice
from tqdm import tqdm
import numpy as np

In [2]:
DATA_DIR = get_project_root() / Path('Data/AB_NYC_2019.CSV')
df = pd.read_csv(DATA_DIR)

In [3]:
df.drop(columns=["last_review", "reviews_per_month", "availability_365"], inplace = True)

In [4]:
dicts = {"central_park": [-73.9665,40.7812],"airport": [-73.8231,40.6657], "empire_state": [-73.9857,40.7484], 
         "time_square": [-73.9855,40.7580],"brooklyn_botanic_garden": [-73.9595,40.6751]}
dist = [*dicts.values()]

In [5]:
df['coor'] = np.array(zip(df.longitude, df.latitude))
df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,coor
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,6,"(-73.97237, 40.64749)"
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2,"(-73.98377, 40.75362)"
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,1,"(-73.9419, 40.80902)"
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,1,"(-73.95976, 40.68514)"
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,1,"(-73.94399, 40.79851)"


In [6]:
API_key = user_config.user_key #enter the key you got from Google. I removed mine here

client = openrouteservice.Client(key=API_key)


In [7]:
value_dict_time = {"central_park": [],"airport": [], "empire_state": [], 
         "time_square": [],"brooklyn_botanic_garden": []}

value_dict_dist = {"central_park": [],"airport": [], "empire_state": [], 
         "time_square": [],"brooklyn_botanic_garden": []}

array_50 =[]
for i in df['coor']:
    array_50.append(i)
    if len(array_50) == 50:
        coordit = array_50 + dist
        array_50 = []
        check = client.distance_matrix(locations=coordit,
                              metrics=['distance', 'duration'],
                              profile='foot-walking',
                              sources=list(range(50)),
                              destinations = list(range(50,55)),
                              validate=False)
        j = 0
        for name in value_dict_time:
            
            value_time = [round(x / 60.0,4) if x is not None else -1 for x in np.transpose(check['durations'])[j]]
            value_dist = [round(x / 100.0,4) if x is not None else -1 for x in np.transpose(check['distances'])[j]]
            
            value_dict_time[name].append(value_time)
            value_dict_dist[name].append(value_dist)
            j += 1
               

C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  stacklevel=1)
C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  stacklevel=1)
C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  stacklevel=1)
C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  stacklevel=1)
C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  stacklevel=1)
C:\Users\sahan\Anaconda3\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  stacklevel=1)


In [8]:
coordit = array_50 + dist
array_50 = []
check = client.distance_matrix(locations=coordit,
                              metrics=['distance', 'duration'],
                              profile='foot-walking',
                              sources=list(range(45)),
                              destinations = list(range(45,50)),
                              validate=False)
j = 0
for name in value_dict_time:
            
    value_time = [round(x / 60.0,4) if x is not None else -1 for x in np.transpose(check['durations'])[j]]
    value_dist = [round(x / 100.0,4) if x is not None else -1 for x in np.transpose(check['distances'])[j]]
            
    value_dict_time[name].append(value_time)
    value_dict_dist[name].append(value_dist)
    j += 1

In [16]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [19]:
q_1 = flatten(value_dict_time["central_park"])
q_2 = flatten(value_dict_time["airport"])
q_3 = flatten(value_dict_time["empire_state"])
q_4 = flatten(value_dict_time["time_square"])
q_5 = flatten(value_dict_time["brooklyn_botanic_garden"])

In [21]:
d_1 = flatten(value_dict_dist["central_park"])
d_2 = flatten(value_dict_dist["airport"])
d_3 = flatten(value_dict_dist["empire_state"])
d_4 = flatten(value_dict_dist["time_square"])
d_5 = flatten(value_dict_dist["brooklyn_botanic_garden"])

In [22]:
df["time_to_central_park"] = q_1
df["time_to_airport"] = q_2
df["time_to_empire_state"] = q_3
df["time_to_time_square"] = q_4
df["time_to_brooklyn_botanic_garden"] = q_5

In [23]:
df["distance_to_central_park"] = d_1
df["distance_to_airport"] = d_2
df["distance_to_empire_state"] = d_3
df["distance_to_time_square"] = d_4
df["distance_to_brooklyn_botanic_garden"] = d_5

In [24]:
df_final = df.drop(columns=['coor', 'latitude','longitude'])

In [25]:
df_final.to_csv('data_time.csv',index=False)